In [ ]:
from tqdm import tqdm

import fastai
from fastai.vision import *
from fastai.callbacks import *
from multiprocessing import Pool
import matplotlib.pyplot as plt
import numpy as np
import PIL
import torch
import torchvision
from torchvision.models import vgg16_bn
from skimage.metrics import structural_similarity as ssim
import os
import sys

from scipy import ndimage
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import Dataset
from torch import is_tensor, FloatTensor,tensor

In [ ]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
print(device)

In [ ]:
from utils.metrics import *

In [ ]:
import time

def measure(fun):
    def wrapper(self):
      start = time.time()
      fun(self)
      end = time.time()
      self.time = end - start

    return wrapper

class AbstractModel:
    def __init__(self):
        self.gt_image = None
        self.lr_image = None
        self.result = None

    def get_name(self) -> str:
        raise NotImplementedError()

    def predict(self):
        raise NotImplementedError()

    def get_result(self) -> np.array:
        raise NotImplementedError()

    def get_metrics(self):
        return [PSNR(np.array(self.result), np.array(self.gt_image)), SSIM(np.array(self.result), np.array(self.gt_image)), self.time]

    def set_input(self, lr_image: PIL.Image, gt_image: PIL.Image):
        self.lr_image = np.array(lr_image)
        self.gt_image = np.array(gt_image)

In [ ]:
base_loss = F.l1_loss

def gram_matrix(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)


class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts
        self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))
              ] + [f'gram_{i}' for i in range(len(layer_ids))]

    def make_features(self, x, clone=False):
        self.m_feat(x)
        return [(o.clone() if clone else o) for o in self.hooks.stored]

    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [base_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)

    def __del__(self): self.hooks.remove()


class UNetModel(AbstractModel):
    def __init__(self):
        super().__init__()
        self.model = load_learner('models')

    def get_name(self) -> str:
        return 'UNet_Model_Nowszy'

    @measure
    def predict(self):
        patch_size = 256
        x = 8
        image = self.lr_image
        temp = np.zeros([int(np.ceil((image.shape[0])/(patch_size-2*x)))*patch_size, int(np.ceil(image.shape[1]/(patch_size-2*x)))*patch_size, 3])
        result = np.zeros((temp.shape[0], temp.shape[1], 3))
        temp[x:image.shape[0]+x, x:image.shape[1]+x] = image
        for i in range(0, temp.shape[0] - patch_size + x +1, patch_size - 2*x):
          for j in range(0, temp.shape[1] - patch_size + x + 1 , patch_size - 2*x):

            j_end = j + patch_size
            im = temp[i:i+patch_size, j:j+patch_size]
            imx = pil2tensor(im ,np.float32)
            pred = self.model.predict(Image(imx))
            pred = np.moveaxis(pred[2].numpy(),0,-1)/255
            pred = np.clip(pred, 0, 1)
            result[i:(i+patch_size-2*x), j:(j+patch_size-2*x)] = pred[x:-x, x:-x]

        self.result =  result[:image.shape[0], :image.shape[1]]

    def get_result(self) -> np.array:
        return self.result

    def set_input(self, lr_image: PIL.Image, gt_image: PIL.Image):
        super().set_input(lr_image.resize((lr_image.size[0]*2, lr_image.size[1]*2), resample=PIL.Image.BILINEAR).convert('RGB'), gt_image)

In [ ]:

class KPNLPModel(AbstractModel):
    def __init__(self):
        super().__init__()
        self.model = KPNLPnetwork().to(device)
        self.model.load_state_dict(torch.load('models/KPNLPresidual.model' , map_location=torch.device(device)))

    def get_name(self) -> str:
        return 'KPNLP_Model'

    @measure
    def predict(self):
        img = np.moveaxis(self.lr_image, -1, 0)
        #print(img.shape)
        lr_image = FloatTensor(img)[None,:]
        lr_image_y = (16+ lr_image[..., 0, :, :]*0.25679 + lr_image[..., 1, :, :]*0.504 + lr_image[..., 2, :, :]*0.09791)/255
        lr_image_y = lr_image_y[None , :, :].to(device)
        lr_image_cb = (128 - 37.945*lr_image[..., 0, :, :]/256 - 74.494*lr_image[..., 1, :, :]/256 + 112.439*lr_image[..., 2, :, :]/256)
        lr_image_cr = (128 + 112.439*lr_image[..., 0, :, :]/256 - 94.154*lr_image[..., 1, :, :]/256 - 18.285*lr_image[..., 2, :, :]/256)
        hr_cb = nn.functional.interpolate(lr_image_cb[None , :,:],scale_factor = 2 , mode='bicubic').detach().numpy()[0,0]
        hr_cr = nn.functional.interpolate(lr_image_cr[None , :,:],scale_factor = 2 , mode='bicubic').detach().numpy()[0,0]
        pom = self.model(lr_image_y)
        pom2 = pom.to('cpu').detach().numpy()[0,0]
        pom2 *= 255
        pom2 = np.clip(pom2 , 0, 255)
        hr_cr = np.clip(hr_cr, 0, 255)
        hr_cb = np.clip(hr_cb , 0, 255)
        r = pom2 + 1.402 *(hr_cr - 128)
        g = pom2 - 0.344136*(hr_cb - 128) - 0.714136 *(hr_cr-128)
        b = pom2 + 1.772* (hr_cb - 128)
        self.result = np.dstack((r,g,b)).astype(np.uint8)



    def get_result(self) -> np.array:
        return self.result

    def set_input(self, lr_image: PIL.Image, gt_image: PIL.Image):
        super().set_input(lr_image, gt_image)

        #print(self.lr_image.shape)
        h_pad = (4 - self.lr_image.shape[0] % 4)%4
        w_pad = (4- self.lr_image.shape[1] % 4)%4
        self.lr_image = np.pad(self.lr_image , ((0, h_pad), (0, w_pad) , (0 ,0)) )
        self.gt_image = np.pad(self.gt_image , ((0, 2*h_pad), (0, 2*w_pad), (0 ,0)) )


In [ ]:
class KPNLPnetwork(nn.Module):
    def __init__(self):
        super(KPNLPnetwork, self).__init__()
        self.kernel = (1.0/100)*torch.tensor([[[[1, 4, 6, 4, 1],[4, 16, 24, 16, 4],[6, 24, 36, 24, 6], [4, 16, 24, 16, 4],[1, 4, 6, 4, 1]]]])
        self.downsample = nn.PixelUnshuffle(4)
        self.conv1a = nn.Conv2d(16 , 64 , 3 , padding=1)
        self.conv1b = nn.Conv2d(64, 64, 3, padding=1)
        self.conv1qa = nn.Conv2d(64, 64, 3, padding=1)
        self.conv1qb = nn.Conv2d(64, 64, 3, padding=1)
        self.conv1ha = nn.Conv2d(16, 64, 3, padding=1)
        self.conv1hb = nn.Conv2d(64, 64, 3, padding=1)
        self.conv1fa = nn.Conv2d(4, 64, 3, padding=1)
        self.conv1fb = nn.Conv2d(64, 64, 3, padding=1)
        self.relu = nn.ReLU()
        
        self.stack1 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack3 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack4 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack5 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack6 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack7 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack8 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack9 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack10 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack11 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack12 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack13 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack14 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack15 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.stack16 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1))
        self.upsample2 = nn.PixelShuffle(2)
        self.upsample4 = nn.PixelShuffle(4)
        self.conv2q = nn.Conv2d(64, 25 , 3 , padding=1)
        self.conv2h = nn.Conv2d(64, 25, 3, padding=1)
        self.conv2f = nn.Conv2d(64, 25, 3, padding=1)
        self.conv3q = nn.Conv2d(25 , 1 , 5, padding='same')
        self.conv3h = nn.Conv2d(25, 1, 5, padding='same')
        self.conv3f = nn.Conv2d(25, 1, 5, padding='same')
        
        self.pyrConv = nn.Conv2d(1 ,1 ,5 , padding="same" , bias=False)
        
        self.pyrConv.weight = nn.Parameter(self.kernel)
        
        self.normalUp = nn.Upsample(scale_factor  = 2 , mode='bicubic')
        self.padLayer = nn.ZeroPad2d(2)
        
    def forward(self, x):
        x = self.normalUp(x) 
        #print(x.shape , x.dtype)
        common = self.downsample(x)
        #print(common.shape , common.dtype)
        common = self.conv1a(common)
        common = self.relu(common)
        #print(common.shape , common.dtype)
        pom = self.stack1(common)
        common = common + pom
        pom = self.stack2(common)
        common = common + pom
        pom = self.stack3(common)
        common = common + pom
        pom = self.stack4(common)
        common = common + pom
        pom = self.stack5(common)
        common = common + pom
        pom = self.stack6(common)
        common = common + pom
        pom = self.stack7(common)
        common = common + pom
        pom = self.stack8(common)
        common = common + pom
        pom = self.stack9(common)
        common = common + pom
        pom = self.stack10(common)
        common = common + pom
        pom = self.stack11(common)
        common = common + pom
        pom = self.stack12(common)
        common = common + pom
        pom = self.stack13(common)
        common = common + pom
        pom = self.stack14(common)
        common = common + pom
        pom = self.stack15(common)
        common = common + pom
        pom = self.stack16(common)
        common = common + pom
        common = self.conv1b(common)
        common = self.relu(common)
        quarter = common
        quarter = self.conv1qa(quarter)
        quarter = self.relu(quarter)
        quarter = self.conv1qb(quarter)
        quarter = self.relu(quarter)
        quarter = self.conv2q(quarter)
        quarter = self.relu(quarter)
        
        half = self.upsample2(common)
        full = self.upsample4(common)
        
        half = self.conv1ha(half)
        half = self.relu(half)
        half = self.conv1hb(half)
        half = self.relu(half)
        half = self.conv2h(half)
        half = self.relu(half)
        

        full = self.conv1fa(full)
        full = self.relu(full)
        full = self.conv1fb(full)
        full = self.relu(full)
        full = self.conv2f(full)
        full = self.relu(full)
        h = x.shape[2]
        w = x.shape[3]
        padded = self.padLayer(x).to(device)
        #padded = nn.functional.pad(x , (2,2,2,2) )
        nq = torch.empty(x.shape[0] , 25, h//4, w//4).to(device)
        nh = torch.empty(x.shape[0] , 25, h//2, w//2).to(device)
        c = torch.empty(x.shape[0] , 25, h, w ).to(device)
        for i in range(h):
            for j in range(w):
                #temp = padded[... , 0, i:i+5 , j:j+5]
                c[...,:,i,j] = torch.flatten(padded[... , 0, i:i+5 , j:j+5] , start_dim=1)
        d = full*c
        e = torch.sum(d , 1, keepdim  = True)

        for i in range(h//2):
            pom_i = i*2
            for j in range(w//2):
                pom_j = j*2
                nh[...,:,i,j] = torch.flatten(padded[... , 0, pom_i:pom_i+5 , pom_j:pom_j+5] , start_dim=1)
        dh = half*nh
        eh = torch.sum(dh , 1, keepdim  = True)

        
        for i in range(h//4):
            pom_i = i*4
            for j in range(w//4):
                pom_j = j*4
                nq[...,:,i,j] = torch.flatten(padded[... , 0, pom_i:pom_i+5 , pom_j:pom_j+5] , start_dim=1)
        dq = quarter*nq
        eq = torch.sum(dq , 1, keepdim  = True)

        eq = self.normalUp(eq)
        eq = self.pyrConv(eq)  #zakomentowane od (2) 
        eh = eh+ eq
        eh = self.normalUp(eh)
        eh = self.pyrConv(eh)  #zakomentowane od (2) 
        e = eh+ e

        
        #e = self.pyrConv(e)       #zakomentowane od (2)
        c.detach()
        eh.detach()
        eq.detach()
        padded.detach()                 
        return e


In [ ]:
def show_image(image):
    image = np.array(image)
    image = np.moveaxis(image, 0, -1)
    plt.imshow(image)
    plt.show()

In [ ]:
def back_projection(y_sr, y_lr, down_kernel='cubic', up_kernel='cubic', sf=None, ds_method='direct'):
    y_sr += image_resize(y_lr - image_resize(y_sr, scale=1.0/sf, output_shape=y_lr.shape, kernel=down_kernel, ds_method=ds_method),
                     scale=sf,
                     output_shape=y_sr.shape,
                     kernel=up_kernel)
    return np.clip(y_sr, 0, 1)

In [ ]:
sys.path.append('MZSR')
sys.path.append('KernelGAN')

from utils import datasets
from image_resize import image_resize
from matplotlib import pyplot as plt

from configs import Config
from data import DataGenerator
from kernelGAN import KernelGAN
from learner import Learner
import torch

class MZSRNetwork(nn.Module):
    def __init__(self):
        super(MZSRNetwork, self).__init__()
        self.stack = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(64, 3, 3, padding='same')
        )
        
    def forward(self, x):
        pred = self.stack(x)
        return x + pred

class MZSRModel(AbstractModel):
    def __init__(self, bicubic=False):
        self.gt_image = None
        self.lr_image = None
        self.result = None
        self.bicubic = bicubic
        self.model = MZSRNetwork().to(device)
        self.conf = Config(device).parse()

    def get_name(self) -> str:
        name = 'MZSR_'
        name += 'bicubic' if self.bicubic else 'kernelGan'
        return name

    @measure
    def predict(self):
        loss_fn = nn.L1Loss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=2.5e-4)
        self.model.load_state_dict(torch.load('models/MZSR.model', map_location=torch.device(device)))

        if self.bicubic:
            kernel = 'cubic'
        else:
            gan = KernelGAN(self.conf)
            learner = Learner()
            data = DataGenerator(self.conf, gan, self.lr_image, device)

            for iteration in range(3000):
                [g_in, d_in] = data.__getitem__(iteration)
                gan.train(g_in, d_in)
                learner.update(iteration, gan)

            kernel = gan.finish()
            print(np.sum(kernel))
            plt.imshow(kernel, cmap='gray')
            plt.show()
        
        training_data = datasets.MZSRMetaTest(
            self.lr_image,
            kernel,
            transform=ToTensor(),
            target_transform=ToTensor()
        )
        
        # show_image(training_data[10][0])
        # show_image(training_data[10][1])
        
        lr_son = image_resize(self.lr_image, scale=1/2, kernel=kernel).astype(np.float32)
        lr_son = image_resize(lr_son, scale=2, kernel='cubic').astype(np.float32)
        lr_son = ToTensor()(lr_son)
        lr_son = lr_son[None]
        # plt.imshow(lr_image)
        # plt.show()
        
        dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
        
        lr_image = ToTensor()(self.lr_image)
        lr_image = lr_image[None]

        self.model.train()
        size = len(dataloader.dataset)
        for i in range(10):
            # for batch, (X, y) in enumerate(dataloader):
                X, y = lr_son.to(device), lr_image.to(device)

                pred = self.model(X)
                loss = loss_fn(pred, y)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
        lr_image = image_resize(self.lr_image, scale=2, kernel='cubic').astype(np.float32)
        lr_image = ToTensor()(lr_image)
        lr_image = lr_image[None]

        self.model.eval()
        with torch.no_grad():
            X = lr_image.to(device)
            
            if not self.bicubic:
                outputs = []
                
                X = torch.moveaxis(lr_image[0], 0, -1)
                for k in range(8):
                    # Rotate 90*k degrees and mirror flip when k>=4
                    test_input = torch.rot90(X, k, (0, 1)) if k < 4 else torch.fliplr(torch.rot90(X, k, (0, 1)))
                    
                    tmp_output = test_input.cpu().detach().numpy()

                    test_input = torch.moveaxis(test_input, -1, 0)[None]
                    # Apply network on the rotated input
                    tmp_output = self.model(test_input.cuda())

                    tmp_output = torch.moveaxis(tmp_output[0], 0, -1)
                    # Undo the rotation for the processed output (mind the opposite order of the flip and the rotation)
                    tmp_output = torch.rot90(tmp_output, -k, (0, 1)) if k < 4 else torch.rot90(torch.fliplr(tmp_output), -k, (0, 1))
                    tmp_output = tmp_output.cpu().detach().numpy()
                    
                    for _ in range(2):
                        tmp_output = back_projection(tmp_output, self.lr_image, sf=2)
                    
                    # save outputs from all augmentations
                    outputs.append(tmp_output)
                
                self.result = np.median(outputs, 0)
                
                for _ in range(2):
                    self.result = back_projection(self.result, self.lr_image, sf=2)
                
                #for _ in range(2):
                #    self.result = back_projection(self.result, self.lr_image, sf=2)
            else:
                pred = self.model(X)
                pred = pred.cpu().detach().numpy()[0]
                self.result = np.moveaxis(pred, 0, -1)     
            
            """if not self.bicubic:
                for _ in range(5):
                    self.result = back_projection(self.result, self.lr_image, sf=2)"""
                    
            self.result = self.result.clip(0, 1)

    def get_result(self) -> np.array:
        return self.result

    def set_input(self, lr_image: np.array, gt_image: np.array):
        super().set_input(lr_image, gt_image)
        self.lr_image = self.lr_image.astype(np.float32) / 255
        self.gt_image = self.gt_image.astype(np.float32) /  255

In [ ]:
class BicubicModel(AbstractModel):
    def __init__(self, bicubic=False):
        self.gt_image = None
        self.lr_image = None
        self.result = None

    def get_name(self) -> str:
        return 'bicubic'

    @measure
    def predict(self):
        self.result = image_resize(self.lr_image, scale=2, kernel='cubic').astype(np.float32).clip(0, 1)
    
    def get_result(self) -> np.array:
        return self.result

    def set_input(self, lr_image: np.array, gt_image: np.array):
        super().set_input(lr_image, gt_image)
        self.lr_image = self.lr_image.astype(np.float32) / 255
        self.gt_image = self.gt_image.astype(np.float32) /  255

In [ ]:
def get_tests(path):
  result = []
  
  with open(path, 'r') as file:
    for line in file:
      while line[-1] == '\n':
        line = line[:-1]

      result.append(line.split(';'))
  
  return result

In [ ]:
def test_on_dataset(path, dataset_lr, dataset_gt, models):
  lista=os.listdir(path/'datasets'/dataset_lr)
  metrics = [open(path/f'results/{dataset_lr}_{i.get_name()}.csv', 'w') for i in models]

  for i in metrics:
      i.write('Name;PSNR;SSIM;time\n')
  
  p_result = path/'results'/dataset_lr

  print(p_result)

  for i in models:
      os.makedirs(p_result/i.get_name(), exist_ok=True)

    
  pbar = tqdm(lista)
  for i in pbar:
      p_lr = f'datasets/{dataset_lr}/{i}'
      p_gt = f'datasets/{dataset_gt}/{i}'

      lr = PIL.Image.open(path/p_lr)
      gt = PIL.Image.open(path/p_gt)

      for j in range(len(models)):
        pbar.set_postfix({'Model': models[j].get_name()})
        
        models[j].set_input(lr, gt)
        models[j].predict()
        result= models[j].get_result()
        temp = p_result/models[j].get_name()/i
        plt.imsave(temp, result)
        img_metrics = models[j].get_metrics()

        temp = str(i)

        for metric in img_metrics:
            temp += f';{metric}'

        metrics[j].write(f'{temp}\n')
        metrics[j].flush()
        os.fsync(metrics[j].fileno())
      torch.cuda.empty_cache()

  for i in metrics:    
      i.close()

In [ ]:
plt.rcParams["figure.figsize"] = (30, 30)

models = [KPNLPModel()]
# models = [UNetModel()]
test_path = Path('test')

tests = get_tests(test_path/'config.csv')

for index, (hr, lr) in enumerate(tests):
  print(f'{index+1}/{len(tests)}: {lr} -> {hr}')

  test_on_dataset(test_path, lr, hr, models)